# Semantic Search

__Outline__
1. Product Search
2. User Search & Analytics
3. Personalized Product Search
4. Recommendations


## Setup

In [1]:
import graphdatascience
graphdatascience.__version__

'1.8'

In [2]:
from graphdatascience import GraphDataScience
from dotenv import load_dotenv
import os
import pandas as pd
from langchain.embeddings import OpenAIEmbeddings, BedrockEmbeddings, SentenceTransformerEmbeddings

In [3]:
load_dotenv('.env', override=True)

True

In [4]:
def load_embedding_model(embedding_model_name: str):
    if embedding_model_name == "openai":
        embeddings = OpenAIEmbeddings()
        dimension = 1536
    elif embedding_model_name == "aws":
        embeddings = BedrockEmbeddings()
        dimension = 1536
    else:
        embeddings = SentenceTransformerEmbeddings(
            model_name="all-MiniLM-L6-v2", cache_folder="/embedding_model"
        )
        dimension = 384
    return embeddings, dimension

In [5]:
embedding_model, dimension = load_embedding_model(os.getenv('EMBEDDING_MODEL'))

In [6]:
# Use Neo4j URI and credentials according to our setup
gds = GraphDataScience(
    os.getenv('NEO4J_URI'),
    auth=(os.getenv('NEO4J_USERNAME'),
          os.getenv('NEO4J_PASSWORD')),
    aura_ds=eval(os.getenv('AURA_DS').title()))

# Necessary if you enabled Arrow on the db - this is true for AuraDS
gds.set_database("neo4j")

In [7]:
gds.version()

'2.4.7+36'

## Product Search
Basic product search using vector index

### Simple Vector Search

In [19]:
#search_prompt = 'denim jeans, loose fit, high-waist'
search_prompt = 'winter sweater with zipper'

In [20]:
query_vector = embedding_model.embed_query(search_prompt)
print(f'query vector length: {len(query_vector)}')
print(f'query vector sample: {query_vector[:10]}')

query vector length: 1536
query vector sample: [-0.015789851900646152, -0.016043451043405262, -0.0007345184150149719, -0.02067496301439282, -0.02582701912369235, 0.013814452414296814, -0.009449886370184712, -0.010043841080765668, 0.0030231627848459975, -0.019406969163242457]


In [21]:
gds.run_cypher('''
CALL db.index.vector.queryNodes("product-text-embeddings", 10, $queryVector)
YIELD node AS product, score
RETURN product.prodName AS prodName,
    product.productTypeName AS productTypeName,
    product.garmentGroupName AS garmentGroupName,
    product.detailDesc AS detailDesc,
    score
''', params={'queryVector': query_vector})

,prodName,productTypeName,garmentGroupName,detailDesc,score
0,Catfish Zip,Sweater,Knitwear,"Soft, textured-knit jumper with a ribbed stand...",0.933365
1,Reena Half Zip sweater,Sweater,Jersey Fancy,Sports top in fast-drying functional fabric wi...,0.925679
2,Allen Sweater,Sweater,Jersey Fancy,Top in sweatshirt fabric made from a cotton bl...,0.925364
3,Winnie top,Sweater,Knitwear,Jumper in a soft rib knit with a turtle neck a...,0.924936
4,Zebra sweater TOP PRODUCT,Sweater,Jersey Fancy,"Top in lightweight, printed sweatshirt fabric ...",0.923741
5,stuvsta,Sweater,"Under-, Nightwear",Fitted jumper in an airy rib knit with a turtl...,0.922499
6,Sheffield Sweater,Sweater,Knitwear,"Turtleneck jumper in a soft, textured knit wit...",0.922328
7,Plum turtleneck,Sweater,Knitwear,Fitted jumper in a soft rib knit with a turtle...,0.921849
8,Maurice sweater,Sweater,Knitwear,Jumper in a soft rib knit with dolman sleeves.,0.921630
9,Janet sweater,Sweater,Knitwear,"Long-sleeved jumper in a soft, fine knit with ...",0.921141
